In [2]:
import pandas as pd

all_games_dataset = pd.read_csv('all_games.csv')

all_games_dataset['release_date'] = pd.to_datetime(all_games_dataset['release_date'])
all_games_dataset.rename(columns={'name': 'Title'}, inplace=True)

all_games_dataset.head()

unique_titles_sample = all_games_dataset['Title'].drop_duplicates().sample(n= 1400)
games_sample = unique_titles_sample.to_frame().merge(all_games_dataset, on= 'Title')
print(str(len(games_sample)))

2079


In [ ]:
import requests

def get_wikipedia_extract(game):
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'titles': game,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }
 
    response = requests.get(url, params=params)
    data = response.json()
    if int(next(iter(data['query']['pages'].keys()))) < 0:
        return ''
    page_extract = next(iter(data['query']['pages'].values()))['extract']
    return page_extract

In [ ]:
wikipedia_extracts = []

for game in all_games_dataset['Title']:
    wikipedia_extracts.append(get_wikipedia_extract(game))

all_games_dataset['Wikipedia'] = wikipedia_extracts
all_games_dataset.head()

In [ ]:
all_games_dataset.to_csv('all_games_clean.csv')
all_games_dataset.head()